In [1]:
import os
import sys
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import f1_score
from datasets import load_dataset, load_metric
from transformers import DataCollatorForSeq2Seq, AdamWeightDecay, \
    TFT5ForConditionalGeneration, T5Tokenizer

c:\Users\andyl\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess_function(examples):
    """ Use tokenizer to preprocess data. """
    
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    prefix = "summarize: "

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=80, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


def download_and_preprocess_data():
    """ Load dataset from HuggingFace and preprocess. """
    
    news_ds = load_dataset("cnn_dailymail", "3.0.0", split="test")

    # Tokenized using preprocess_function
    tokenized_news = news_ds.map(preprocess_function, batched=True)

    return tokenized_news

In [3]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

optimizer = AdamWeightDecay(
    learning_rate=2e-5, 
    weight_decay_rate=0.01
)

model = TFT5ForConditionalGeneration.from_pretrained("t5_small_news")
model.compile(optimizer=optimizer)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    model=model, 
    return_tensors="tf",
)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5_small_news.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [4]:
tokenized_news = download_and_preprocess_data()
tokenized_news

Reusing dataset cnn_dailymail (C:\Users\andyl\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Loading cached processed dataset at C:\Users\andyl\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234\cache-55793dd6c08d73ff.arrow


Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11490
})

In [8]:
test_ds = tokenized_news.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

In [9]:
def compute_metrics(metric, pred, actual):
    """ Compute the model's rouge performance on an instance. """

    metric.add(predictions=pred, references=actual)
    final_score = metric.compute()
    
    return final_score

In [12]:
metric = load_metric('rouge')
result = [[] for x in range(3)]

cnt = 0
for item in test_ds:
    article = item['input_ids']
    actual = item['labels']
    
    pred = model.generate(
        do_sample=True,
        input_ids=article,
        # min_length=56,
        max_length=80,
        temperature=0.8, 
        top_k=45,
        no_repeat_ngram_size=3,
        num_beams=5,
        early_stopping=True
    )

    rouge_score = compute_metrics(metric, pred, actual)
    rouge1 = 100 * rouge_score['rouge1'][1][2]
    rouge2 = 100 * rouge_score['rouge2'][1][2]
    rougeL = 100 * rouge_score['rougeL'][1][2]

    cnt += 1 
    if cnt % 25 == 0:
        print(f'Round: {cnt * 4}')

    result[0].append(rouge1)
    result[1].append(rouge2)
    result[2].append(rougeL)

TypeError: can only concatenate str (not "int") to str

In [47]:
result_df = pd.DataFrame(data = np.array(result).T, columns = ['Rouge1','Rouge2','RougeL'])
result_df

,Rouge1,Rouge2,RougeL
0,43.525180,19.639640,26.258993
1,39.583333,18.138042,23.076923
2,37.738854,17.384370,23.248408
3,40.816327,18.568995,22.108844
4,39.673913,12.522686,19.746377
...,...,...,...
96,44.463087,27.226891,30.201342
97,36.858974,16.853933,20.512821
98,46.626984,23.856859,29.761905
99,45.270270,20.473773,27.364865


In [48]:
result_df.describe()

,Rouge1,Rouge2,RougeL
count,101.000000,101.000000,101.000000
mean,41.122402,18.848548,23.775518
std,3.732857,3.693768,3.519158
min,28.965517,11.413969,17.346939
25%,39.137931,16.695652,21.656051
50%,41.134752,18.568995,23.310811
75%,43.387097,20.582524,25.178571
max,56.211180,38.102644,42.236025


## Play around with text summarization

In [13]:
article = """ 
NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."
Moninder Singh Pandher was sentenced to death by a lower court in February.
The teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.
The Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.
Pandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.
The high court upheld Koli's death sentence, Kochar said.
"""

In [14]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenized_input = tokenizer("summarize: " + article, max_length=1024, truncation=True, return_tensors='tf')

pred = model.generate(
    do_sample=True,
    input_ids=tokenized_input['input_ids'],
    min_length=56,
    max_length=128,
    temperature=0.8, 
    top_k=45,
    no_repeat_ngram_size=3,
    num_beams=5,
    early_stopping=True
)

pred_sentence = tokenizer.decode(pred[0], skip_special_tokens=True)

print(f"original = {article}\n")
print(f"pred = {pred_sentence}\n")

original =  
NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."
Moninder Singh Pandher was sentenced to death by a lower court in February.
The teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.
The Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.
Pandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.
The high court upheld Koli's death sentence, Kochar said.


pred = Moninder Singh Pandher was one of 19 victims in one of the most gruesome serial killings in India. Pandhe was sentenced to death by a lower court in February. The teen was one among 19 victims -- children and young women.

